# Extracting scene graphs for 3RScan scenes dataset

In [15]:
PATH_TO_RELATIONS = 'datasets/3RScan/data/3RScan/relationships.json'
PATH_TO_SCENE_FOLDERS = 'datasets/3RScan/data/3RScan'

scene_id = '4acaebcc-6c10-2a2a-858b-29c7e4fb410d'

1. Read the contents of the "relationships.json" file to extract the relationships between objects in the scenes

In [30]:
import json

with open(PATH_TO_RELATIONS) as f:
    relationships_data = json.load(f)

scene_relationships = relationships_data['scans']

# Find the relationships for the scene with the given id
scene_relationships = [r for r in scene_relationships if r['scan'] == scene_id]

print('Found {} relationships dictionary for scene {}'.format(len(scene_relationships), scene_id))
scene_relationships = scene_relationships[0]['relationships']

# Print the first 10 relationships
n = 10
print(f"Found {len(scene_relationships)} relationships for scene {scene_id}, here are the first {n}:")
for i, r in enumerate(scene_relationships[:n]):
    print(f"Relationship #{i+1:<2}: {r}")

Found 1 relationships dictionary for scene 4acaebcc-6c10-2a2a-858b-29c7e4fb410d
Found 310 relationships for scene 4acaebcc-6c10-2a2a-858b-29c7e4fb410d, here are the first 10:
Relationship #1 : [2, 1, 14, 'attached to']
Relationship #2 : [3, 1, 14, 'attached to']
Relationship #3 : [4, 3, 14, 'attached to']
Relationship #4 : [5, 4, 15, 'standing on']
Relationship #5 : [6, 4, 15, 'standing on']
Relationship #6 : [7, 4, 15, 'standing on']
Relationship #7 : [8, 4, 15, 'standing on']
Relationship #8 : [11, 3, 14, 'attached to']
Relationship #9 : [13, 4, 15, 'standing on']
Relationship #10: [14, 1, 14, 'attached to']


2. Read the contents of each "semseg.v2.json" file in the scene folders to extract object information

In [26]:
import os

scene_graphs = []
# relation_types = set()
# object_ids = set()

file_path = os.path.join(PATH_TO_SCENE_FOLDERS, scene_id, 'semseg.v2.json')
with open(file_path) as f:
    scene_data = json.load(f)

objects = scene_data['segGroups']
# TODO: use index instead of objectId as key?
scene_objects = [(obj['objectId'], obj['label']) for obj in objects]

scene_graphs.append(scene_objects)

n = 3
print(f"Found {len(scene_objects)} objects in the scene, here are the first {n}:")
for i, obj in enumerate(scene_objects[:n]):
    print(f"Object #{i+1}: {obj}")

Found 32 objects in the scene, here are the first 3:
Object #1: (17, 'beanbag')
Object #2: (18, 'beanbag')
Object #3: (19, 'table')


3. Create a mapping of relation types to integer values

In [33]:
relation_type_mapping = {}
for relation in scene_relationships:
    relation_type = relation[3]
    if relation_type not in relation_type_mapping:
        relation_type_mapping[relation_type] = len(relation_type_mapping)

n = 6
print(f"Found {len(relation_type_mapping)} relation types, here are the first {n}:")
for i, relation_type in enumerate(list(relation_type_mapping.keys())[:n]):
    print(f"Relation type #{i+1}: {relation_type}")


Found 25 relation types, here are the first 6:
Relation type #1: attached to
Relation type #2: standing on
Relation type #3: connected to
Relation type #4: cover
Relation type #5: lying on
Relation type #6: leaning against


4. TODO

In [34]:
N = []
E = [[], []]
R = []

for scene_objects in scene_graphs:
    node_mapping = {obj[0]: i for i, obj in enumerate(scene_objects)}
    
    for obj in scene_objects:
        N.append(obj)
    
    for relation in scene_relationships:
        source_id = relation[0]
        target_id = relation[1]
        relation_type = relation[3]
        
        if source_id in node_mapping and target_id in node_mapping:
            source_idx = node_mapping[source_id]
            target_idx = node_mapping[target_id]
            
            E[0].append(source_idx)
            E[1].append(target_idx)
            R.append(relation_type_mapping[relation_type])


In [35]:
import numpy as np

N = np.array(N, dtype=object)
E = np.array(E)
R = np.array(R)

In [44]:
print(f"Dimensions check: N: {N.shape}, E: {E.shape}, R: {R.shape}")

n = 20

print(f"Here are the first {n} nodes:")
print(N[:n].T)

print(f"Here are the first {n} edges:")
print(E[:, :n])

print(f"Here are the first {n} relations:")
print(R[:n])

Dimensions check: N: (32, 2), E: (2, 310), R: (310,)
Here are the first 20 nodes:
[[17 18 19 20 21 22 23 24 25 26 27 28 29 30 33 100 1 2 3 4]
 ['beanbag' 'beanbag' 'table' 'cushion' 'cushion' 'sofa' 'cushion' 'tv'
  'commode' 'ceiling' 'plant' 'window' 'light' 'light' 'window' 'window'
  'floor' 'wall' 'wall' 'windowsill']]
Here are the first 20 edges:
[[17 18 19 20 21 22 23 26 28 29 30 31  2  3  4  5  6  7  8  9]
 [16 16 18 19 19 19 19 18 19 16 18 16 16  5  5 16  4  8 16 17]]
Here are the first 20 relations:
[0 0 0 1 1 1 1 0 1 0 2 1 1 3 4 1 5 1 1 0]
